In [1]:
import os
import sys
from datetime import datetime

import yaml
import numpy as np
import pandas as pd


os.chdir('..')
sys.path.append(os.getcwd())

# Initial configurations
config_filepath = 'config.yml'

with open(config_filepath, 'r') as f:
    config = yaml.safe_load(f)

# Modeling

In [2]:
abt = pd.read_csv(config['filepaths']['abt'])

In [3]:
abt

,Age,SibSp,Parch,Fare,Survived,Pclass,Sex,Embarked
0,22,1,0,7.250,0,3,male,S
1,38,1,0,50.000,1,1,female,C
2,26,0,0,7.925,1,3,female,S
3,35,1,0,50.000,1,1,female,S
4,35,0,0,8.050,0,3,male,S
...,...,...,...,...,...,...,...,...
709,39,0,5,29.125,0,3,female,Q
710,27,0,0,13.000,0,2,male,S
711,19,0,0,30.000,1,1,female,S
712,26,0,0,30.000,1,1,male,C


## 1 Establishing a Baseline

In [21]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

In [4]:
label = 'Fare'

In [5]:
naive_prediction = abt[label].mean()

In [6]:
naive_prediction

22.831097759103642

In [13]:
mean_absolute_error(abt[label], np.ones(len(abt))*naive_prediction)

14.119048114147619

In [16]:
root_mean_squared_error(abt[label], np.ones(len(abt))*naive_prediction)

16.201047279407675

## 2 Data Segregation

In [17]:
train_size = 0.80
random_state = 1337

train_data = abt.sample(frac=0.80, random_state=random_state)
test_data = abt.loc[~abt.index.isin(train_data.index)]

## 3 Model Training

In [18]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [19]:
train_dataset = TabularDataset(train_data)
test_dataset = TabularDataset(test_data)

### Automatic

In [23]:
metric = 'rmse'
presets = 'best_quality'
time_limit = 60 # in seconds
save_path = f'models/{datetime.now().strftime("%Y%m%d-%H%M")}'
verbosity = 2

predictor = TabularPredictor(label=label, eval_metric=metric, path=save_path)
predictor = predictor.fit(
    train_dataset, time_limit=time_limit, presets=presets, verbosity=verbosity,
    dynamic_stacking=False
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.17
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #52-Ubuntu SMP PREEMPT_DYNAMIC Thu Dec  5 13:09:44 UTC 2024
CPU Count:          32
Memory Avail:       234.97 GB / 251.52 GB (93.4%)
Disk Space Avail:   3801.79 GB / 71394.83 GB (5.3%)
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "/home/llorenzo/year-2025/Acads/msds2025-ft/adhoc/insta-ml/models/20250425-1003"
Train Data Rows:    571
Train Data Columns: 7
Label Column:       Fare
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (50.0, 0.0, 22.92221, 16.34059)
	If 'regression' is not the correct pro

### Manual

In [24]:
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

In [25]:
get_hyperparameter_config('default')

{'NN_TORCH': {},
 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
  {},
  {'learning_rate': 0.03,
   'num_leaves': 128,
   'feature_fraction': 0.9,
   'min_data_in_leaf': 3,
   'ag_args': {'name_suffix': 'Large',
    'priority': 0,
    'hyperparameter_tune_kwargs': None}}],
 'CAT': {},
 'XGB': {},
 'FASTAI': {},
 'RF': [{'criterion': 'gini',
   'ag_args': {'name_suffix': 'Gini',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'entropy',
   'ag_args': {'name_suffix': 'Entr',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'squared_error',
   'ag_args': {'name_suffix': 'MSE',
    'problem_types': ['regression', 'quantile']}}],
 'XT': [{'criterion': 'gini',
   'ag_args': {'name_suffix': 'Gini',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'entropy',
   'ag_args': {'name_suffix': 'Entr',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'squared_error',
   'ag_args': {'name_suffix': 'MSE',
    'problem_ty

In [26]:
from autogluon.common import space

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'dropout_prob': space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'scheduler': 'local',
    'searcher': search_strategy,
}  # Refer to TabularPredictor.fit docstring for all valid values

In [28]:
metric = 'rmse'
presets = 'best_quality'
time_limit = 30 # in seconds
save_path = f'models/{datetime.now().strftime("%Y%m%d-%H%M")}'
verbosity = 2

predictor = TabularPredictor(label=label, eval_metric=metric, path=save_path)
predictor = predictor.fit(
    train_dataset, time_limit=time_limit, presets=presets, verbosity=verbosity,
    dynamic_stacking=False, hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No model was trained during hyperparameter tuning NeuralNetTorch_BAG_L1... Skipping this model.
Fitting model: WeightedEnsemble_L2 ... Training model for up to 29.90s of the -2.58s of remaining time.
	Ensemble Weights: {'LightGBM_BAG_L1/T2': 1.0}
	-7.399	 = Validation score   (-root_mean_squared_error)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 32.63s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 2809.9 rows/s (72 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/llorenzo/year-2025/Acads/msds2025-ft/adhoc/insta-ml/models/20250425-1005")


## 4 Model Performance

In [29]:
predictor.predict(test_dataset)

0      11.512443
5      27.610825
7      18.173141
10     40.906746
22     29.951815
         ...    
694    15.731112
696    47.960033
705     8.671460
707    14.283507
712    47.574890
Name: Fare, Length: 143, dtype: float32

In [30]:
predictor.evaluate(test_dataset)

{'root_mean_squared_error': -6.2819366910350585,
 'mean_squared_error': -39.462728590172496,
 'mean_absolute_error': -3.951785368043059,
 'r2': 0.839611170134093,
 'pearsonr': 0.9176679338220373,
 'median_absolute_error': -2.039966583251953}

In [31]:
predictor.leaderboard(test_dataset)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1/T1,-6.198317,-7.464360,root_mean_squared_error,0.062747,0.028695,3.303470,0.062747,0.028695,3.303470,1,True,1
1,LightGBM_BAG_L1/T2,-6.281937,-7.398975,root_mean_squared_error,0.037366,0.025535,3.188992,0.037366,0.025535,3.188992,1,True,2
2,WeightedEnsemble_L2,-6.281937,-7.398975,root_mean_squared_error,0.040894,0.026236,3.193206,0.003527,0.000701,0.004214,2,True,5
3,LightGBM_BAG_L1/T3,-6.357021,-8.033043,root_mean_squared_error,0.037823,0.023102,3.364038,0.037823,0.023102,3.364038,1,True,3
4,LightGBM_BAG_L1/T4,-9.999238,-11.200052,root_mean_squared_error,0.093568,0.025567,3.283561,0.093568,0.025567,3.283561,1,True,4


In [32]:
predictor.model_best

'WeightedEnsemble_L2'

In [33]:
predictor.model_names()

['LightGBM_BAG_L1/T1',
 'LightGBM_BAG_L1/T2',
 'LightGBM_BAG_L1/T3',
 'LightGBM_BAG_L1/T4',
 'WeightedEnsemble_L2']

In [35]:
predictor.evaluate(model='LightGBM_BAG_L1/T4', data=test_dataset)

{'root_mean_squared_error': -9.999238090608905,
 'mean_squared_error': -99.98476239268403,
 'mean_absolute_error': -8.65021348319954,
 'r2': 0.5936307595167931,
 'pearsonr': 0.9179633960924988,
 'median_absolute_error': -8.356958026123046}